In [ ]:
!pip install chess

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 64.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for chess: filename=chess-1.11.2-py3-none-any.whl size=147775 sha256=5adb57cb052040489ac21e21ec552876f632f30c12371409e4dd4456b2bf7583
  Stored in directory: /root/.cache/pip/wheels/fb/5d/5c/59a62d8a695285e59ec9c1f66add6f8a9ac4152499a2be0113
Successfully built chess


In [ ]:
!pip install chess.pgn

ERROR: Could not find a version that satisfies the requirement chess.pgn (from versions: none)
ERROR: No matching distribution found for chess.pgn


In [20]:
import io, os, re, json
from pathlib import Path
from typing import List, Tuple, Literal
import chess, chess.pgn
from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive
from google.colab import auth as colab_auth
from oauth2client.client import GoogleCredentials

FOLDER_ID = "1lxS3GpYvDLU_1a9oqzIRlNmJUSYbmGZp"
local_dir = Path("./Multiviews3")
local_dir.mkdir(exist_ok=True)

def download_folder_all(folder_id: str, dst: Path) -> None:
    colab_auth.authenticate_user()
    gauth = GoogleAuth()
    gauth.credentials = GoogleCredentials.get_application_default()
    drive = GoogleDrive(gauth)

    dst.mkdir(parents=True, exist_ok=True)

    def _recurse(fid: str, out_dir: Path):
        out_dir.mkdir(parents=True, exist_ok=True)
        q = f"'{fid}' in parents and trashed=false"
        for f in drive.ListFile({'q': q}).GetList():
            if f['mimeType'] == 'application/vnd.google-apps.folder':
                _recurse(f['id'], out_dir / f['title'])
            else:
                out_path = out_dir / f['title']
                if out_path.exists() and out_path.stat().st_size > 0:
                    continue
                f.GetContentFile(str(out_path))

    _recurse(folder_id, dst)

if not any(local_dir.iterdir()):
    download_folder_all(FOLDER_ID, local_dir)
else:
    print("Skipping download: files already present.")

Mode = Literal["multiview3", "overhead"]

def natural_key(s: str):
    return [int(t) if t.isdigit() else t.lower() for t in re.findall(r"\d+|\D+", s)]

def read_images(folder: Path, exts: Tuple[str, ...]=(".png",".jpg",".jpeg",".webp",".bmp")) -> List[str]:
    imgs = [f.name for f in folder.iterdir() if f.is_file() and f.suffix.lower() in exts]
    imgs.sort(key=natural_key)
    return imgs

def pgn_to_fens(pgn_text: str, include_start: bool = False) -> List[str]:
    s = re.sub(r'\]\s*\[', ']\n[', pgn_text.strip())
    s = re.sub(r'\]\s*(?=\d+\.)', ']\n\n', s)

    game = chess.pgn.read_game(io.StringIO(s))
    if game is None:
        raise ValueError("Could not parse PGN.")

    board = game.board()
    fens = [board.fen()] if include_start else []

    moves = list(game.mainline_moves())
    if not moves:
        raise ValueError("Parsed PGN but found no moves. Ensure tags and moves are separated by a blank line.")

    for mv in moves:
        board.push(mv)
        fens.append(board.fen())
    return fens

def assign_labels(image_names: List[str], fens: List[str], mode: Mode, start_ply: int=0):
    labels = []
    if mode == "multiview3":
        for i, img in enumerate(image_names):
            fen_idx = start_ply + (i // 4)
            if fen_idx >= len(fens):
                fen_idx = len(fens) - 1
            labels.append({"image": img, "gt_fen": fens[fen_idx]})
    elif mode == "overhead":
        for i, img in enumerate(image_names):
            fen_idx = start_ply + i
            if fen_idx >= len(fens):
                fen_idx = len(fens) - 1
            labels.append({"image": img, "gt_fen": fens[fen_idx]})
    else:
        raise ValueError("mode must be 'multiview3' or 'overhead'")
    return labels

pgn_text = r'''[Event "Hoogovens Group A"] [Site "Wijk aan Zee NED"] [Date "1999.01.20"] [Round "4"] [White "Garry Kasparov"] [Black "Veselin Topalov"] [Result "1-0"] [WhiteElo "2812"] [BlackElo "2700"] [Variant "Standard"] [ECO "B07"] [Opening "Pirc Defense"] [StudyName "Topalov vs Kasparov/1999"] [ChapterName "Capítulo 1"] [Annotator "https://lichess.org/@/emejor2003"] [ChapterURL "https://lichess.org/study/UU8bF5Ap/Q96G90LR"] 1. e4 d6 2. d4 Nf6 3. Nc3 g6 4. Be3 Bg7 5. Qd2 { [%cal Ge3h6,Gg7h6,Gd2h6] } 5... c6 6. f3 b5 7. Nge2 Nbd7 8. Bh6 Bxh6 9. Qxh6 Bb7 10. a3 { [%cal Gb5b4,Ga3b4] } 10... e5 11. O-O-O Qe7 12. Kb1 { [%cal Ge2c1,Gc1b3] } 12... a6 13. Nc1 { [%cal Gc1b3,Gb3d4,Gb3a5] } 13... O-O-O 14. Nb3 exd4 15. Rxd4 (15. Nxd4) 15... c5 16. Rd1 Nb6 17. g3 { [%cal Gf1h3,Gh3c8] } 17... Kb8 { [%cal Gf1h3,Gh3c8] } 18. Na5 Ba8 19. Bh3 d5 20. Qf4+ Ka7 21. Rhe1 { [%cal Ge1e7,Ge4d5] } 21... d4 22. Nd5 Nbxd5 23. exd5 { [%cal Ge1e7] } 23... Qd6 24. Rxd4 (24. Re7+) 24... cxd4 25. Re7+ { [%cal Gd6e7,Gf4d4,Ga7b8,Gd4b6,Ga8b7,Ga5c6,Gb8a8,Gb6a7] } 25... Kb6 (25... Qxe7 26. Qxd4+ Kb8 27. Qb6+ Bb7 28. Nc6+ Ka8 29. Qa7#) 26. Qxd4+ Kxa5 27. b4+ Ka4 28. Qc3 { [%csl Gb4,Ga3,Gb5,Ga5,Gb3][%cal Gc3b3] } 28... Qxd5 { [%cal Gd5b3,Gc3b3] } 29. Ra7 { [%cal Ga7a6] } 29... Bb7 { [%cal Gb7a6] } 30. Rxb7 { [%cal Gc3f6,Gf6e8,Gf6h5,Gb7a7,Ga7a6] } 30... Qc4 (30... Nh5) (30... Ne8 31. Ra7) 31. Qxf6 Kxa3 32. Qxa6+ Kxb4 33. c3+ Kxc3 34. Qa1+ Kd2 35. Qb2+ Kd1 36. Bf1 { [%csl Gc4][%cal Gc4f1,Gb2c2,Gd1e1,Gb7e7] } 36... Rd2 (36... Qxf1 37. Qc2+ Ke1 38. Re7+ Qe2 39. Qxe2#) 37. Rd7 Rxd7 38. Bxc4 bxc4 39. Qxh8 Rd3 40. Qa8 c3 (40... h5) 41. Qa4+ Ke1 42. f4 f5 43. Kc1 Rd2 (43... h6) (43... h5 44. Qe8+ Kf2) (43... Rd4) 44. Qa7 { 1-0 Black resigns. } 44... Ke2 { [%csl Gh7,Gg6,Gf5][%cal Ga7h7,Gh7g6,Gg6f5,Gf4f5,Gg3g4,Gg4g5,Gh2h4,Gh4h5] } (44... h6 45. Qg1+ Ke2 46. Qg2+ Ke3) 1-0
'''
fens = pgn_to_fens(pgn_text)
imgs = read_images(local_dir)
labels = assign_labels(imgs, fens, mode="multiview3", start_ply=0)

with open(local_dir / "labels.json", "w", encoding="utf-8") as f:
    json.dump(labels, f, ensure_ascii=False, indent=2)

print(f"Found {len(imgs)} images in {local_dir.resolve()}")
print(f"Wrote {len(labels)} labels -> {local_dir/'labels.json'}")
print(f"Total FENs from PGN: {len(fens)}")


Found 352 images in /content/Multiviews3
Wrote 352 labels -> Multiviews3/labels.json
Total FENs from PGN: 88


In [ ]:
from pathlib import Path
import shutil

def empty_folder(path: Path) -> None:
    """Delete all files and subfolders inside `path` (keep the folder itself)."""
    if not path.exists() or not path.is_dir():
        return
    for p in path.iterdir():
        if p.is_dir():
            shutil.rmtree(p)
        else:
            p.unlink()

# call it
empty_folder(local_dir)